In [1]:
!pip install torch
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import torch, os, re, pandas as pd, json
import torch
# from sklearn.model_selection import train_test_split
# from transformers import DataCollatorForLanguageModeling, DataCollatorWithPadding, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoConfig

# from transformers import get_linear_schedule_with_warmup

# import pandas as pd
# from transformers import GPT2LMHeadModel, GPT2Tokenizer
# import transformers
# import numpy as np
# import random
# import torch
# from torch.utils.data import Dataset, DataLoader
# from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
# from tqdm import tqdm, trange
# import torch.nn.functional as F
# import csv

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
!git clone https://github.com/alontalmor/pytorch-transformers.git

fatal: destination path 'pytorch-transformers' already exists and is not an empty directory.


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
# head_model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)

In [6]:
# # %cd ..
# !pwd


In [7]:
def truncate_str_for_sen_embed(text_list, max_len=384): #ref christian
    shortened_texts = []
    for text in text_list:
        start = 0
        end = max_len
        while True:
            if len(text) <= max_len:
                shortened_texts.append(text)
                break
            shortened_texts.append(text[:max_len])
            text = text[max_len:]
            
    return shortened_texts

In [8]:
with open('/home/ubuntu/Darwin_Gatsby/gatsby_raw.txt', 'r') as file:
    gatsby = file.read().replace('\n', ' ')

with open('/home/ubuntu/Darwin_Gatsby/origin_of_species_raw.txt', 'r') as file:
    origin = file.read().replace('\n', ' ')

In [9]:
gatsby_shortened = truncate_str_for_sen_embed([gatsby], max_len=384)

origin_shortened = truncate_str_for_sen_embed([origin], max_len=384)


# ref https://stackoverflow.com/questions/8689795/how-can-i-remove-non-ascii-characters-but-leave-periods-and-spaces
import re
import string
from collections import Counter
from collections import defaultdict
origin_short_clean = []
for sen_o in origin_shortened:
    sen_o = re.sub(r'^\s*|\s\s*', ' ', sen_o).strip().rstrip().lstrip()
    printable = set(string.printable)
    sen_o = filter(lambda x: x in printable, sen_o)
    sen_o = ''.join(filter(lambda x: x in printable, sen_o))
    origin_short_clean.append(sen_o)
gatsby_short_clean = []
for sen_g in gatsby_shortened:
    sen_g = re.sub(r'^\s*|\s\s*', ' ', sen_g).strip().rstrip().lstrip()
    printable = set(string.printable)
    sen_g = filter(lambda x: x in printable, sen_g)
    sen_g = ''.join(filter(lambda x: x in printable, sen_g))
    gatsby_short_clean.append(sen_g)
    

In [10]:
from io import StringIO
from html.parser import HTMLParser

# ref https://stackoverflow.com/questions/753052/strip-html-from-strings-in-python
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()
# cleaned = test_text_pos.replace('<br /><br />', ' ')
# print(cleaned)

def clean_imdb(review_list):
    for review in range(len(review_list)):
        cleaned_review = strip_tags(review_list[review])
        review_list[review] = cleaned_review

In [11]:
import os

pos_path = "/home/ubuntu/pos/"
neg_path = "/home/ubuntu/neg/"
home_path = "/home/ubuntu/"
os.chdir(home_path)
pos_token_list = []
pos_text_list = []
neg_token_list = []
neg_text_list = []

os.chdir(pos_path)
  
count = 0
len_sum = 0
for file in os.listdir():
    with open(pos_path+file, 'r') as f:
        pos_text = f.read()
        # put into text list
        pos_text_list.append(pos_text)
        #tokenize and put into token list
        #pos_token_list.append(base_tokenizer.encode(pos_text, return_tensors = 'pt'))
        len_sum += len(pos_text)
        count += 1
os.chdir(neg_path)
for file in os.listdir():
    with open(neg_path+file, 'r') as f:
        neg_text = f.read()
        # put into text list
        neg_text_list.append(neg_text)
        #tokenize and put into token list
        #neg_token_list.append(base_tokenizer.encode(neg_text_list, return_tensors = 'pt'))
        len_sum += len(neg_text)
        count += 1
os.chdir(home_path)

In [12]:
clean_imdb(pos_text_list)
clean_imdb(neg_text_list)

In [13]:
print(f"Average len:{len_sum/count}")

Average len:1325.06964


In [14]:
# df_pos_tokens = pd.DataFrame(pos_token_list, columns = ['tokens'])
# print(df_pos_tokens)
pd.options.display.max_colwidth = 1024
df_pos_text = pd.DataFrame(pos_text_list, columns = ['text'])
# print(df_pos_text)
df_pos_text_small = df_pos_text[:8]
# print(df_pos_text_small)

In [15]:
# !pwd
# df_pos_text[0:1]

In [16]:
# test_txt_file = np.savetxt(r'/content/FineTuneText/ImdbPos.txt', df_pos_text.values, newline='\n', fmt='%s')
# df_pos_text.to_csv('test_pos.txt', sep='\n', index=False)


In [17]:
#gatsby
pd.options.display.max_colwidth = 1024
df_gatsby_text = pd.DataFrame(gatsby_short_clean, columns = ['text'])


In [18]:
#origin
pd.options.display.max_colwidth = 1024
df_origin_text = pd.DataFrame(origin_short_clean, columns = ['text'])

In [19]:
with open("test_gatsby_loop.txt", "w") as f_out:
    f_out.write("\n".join(df_gatsby_text["text"]))

In [20]:
with open("test_origin_loop.txt", "w") as f_out:
    f_out.write("\n".join(df_origin_text["text"]))

In [21]:
with open("test_pos_loop.txt", "w") as f_out:
    f_out.write("\n".join(df_pos_text["text"]))

In [22]:
# !export TRAIN_FILE=/content/FineTuneText/ImdbPos.txt
#     --overwrite_cache \
!python  /home/ubuntu/pytorch-transformers/examples/run_lm_finetuning.py \
    --output_dir=FTGatsby_model \
    --model_type=gpt2 \
    --model_name_or_path=gpt2 \
    --do_train \
    --learning_rate=2e-4 \
    --save_steps=116 \
    --warmup_steps=50 \
    --gradient_accumulation_steps=1 \
    --overwrite_output_dir \
    --train_data_file=/home/ubuntu/test_gatsby_loop.txt \
    --num_train_epochs=10

--------------------------------------------------------------------------
detected:

  Local host:            130-61-76-112
  Device name:           mlx5_0
  Device vendor ID:      0x02c9
  Device vendor part ID: 4122

Default device parameters will be used, which may result in lower
performance.  You can edit any of the files specified by the
btl_openib_device_param_files MCA parameter to set values for your
device.

NOTE: You can turn off this warning by setting the MCA parameter
      btl_openib_warn_no_device_params_found to 0.
--------------------------------------------------------------------------
--------------------------------------------------------------------------
No OpenFabrics connection schemes reported that they were able to be
used on a specific port.  As such, the openib BTL (OpenFabrics
support) will be disabled for this port.

  Local host:           130-61-76-112
  Local device:         mlx5_0
  Local port:           1
  CPCs attempted:       udcm
-------------

In [23]:
# !export TRAIN_FILE=/content/FineTuneText/ImdbPos.txt
#     --overwrite_cache \
!python  /home/ubuntu/pytorch-transformers/examples/run_lm_finetuning.py \
    --output_dir=FTOrigin_model \
    --model_type=gpt2 \
    --model_name_or_path=gpt2 \
    --do_train \
    --learning_rate=2e-4 \
    --save_steps=116 \
    --warmup_steps=50 \
    --gradient_accumulation_steps=1 \
    --overwrite_output_dir \
    --train_data_file=/home/ubuntu/test_origin_loop.txt \
    --num_train_epochs=10

--------------------------------------------------------------------------
detected:

  Local host:            130-61-76-112
  Device name:           mlx5_0
  Device vendor ID:      0x02c9
  Device vendor part ID: 4122

Default device parameters will be used, which may result in lower
performance.  You can edit any of the files specified by the
btl_openib_device_param_files MCA parameter to set values for your
device.

NOTE: You can turn off this warning by setting the MCA parameter
      btl_openib_warn_no_device_params_found to 0.
--------------------------------------------------------------------------
--------------------------------------------------------------------------
No OpenFabrics connection schemes reported that they were able to be
used on a specific port.  As such, the openib BTL (OpenFabrics
support) will be disabled for this port.

  Local host:           130-61-76-112
  Local device:         mlx5_0
  Local port:           1
  CPCs attempted:       udcm
-------------

In [24]:
# with open("test_pos_loop_small.txt", "w") as f_out:
#     f_out.write(df_pos_text_small["text"].to_string(index=False))

In [25]:
torch.cuda.device_count()

8

In [26]:
# !python3 -m pip install --upgrade pip
# !python -m pip install --upgrade pandas

In [27]:
# num_gpus = os.environ['CUDA_VISIBLE_DEVICES'].split(',').__len__()
# os.environ['CUDA_VISIBLE_DEVICES'] = ','.join(f'{i}' for i in range(8))
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [28]:
# !export TRAIN_FILE=/content/FineTuneText/ImdbPos.txt
#     --overwrite_cache \
!python  /home/ubuntu/pytorch-transformers/examples/run_lm_finetuning.py \
    --output_dir=ImdbPOS_model \
    --model_type=gpt2 \
    --model_name_or_path=gpt2 \
    --do_train \
    --learning_rate=2e-4 \
    --save_steps=116 \
    --warmup_steps=50 \
    --gradient_accumulation_steps=1 \
    --overwrite_output_dir \
    --train_data_file=/home/ubuntu/test_pos_loop.txt \
    --num_train_epochs=10

--------------------------------------------------------------------------
detected:

  Local host:            130-61-76-112
  Device name:           mlx5_0
  Device vendor ID:      0x02c9
  Device vendor part ID: 4122

Default device parameters will be used, which may result in lower
performance.  You can edit any of the files specified by the
btl_openib_device_param_files MCA parameter to set values for your
device.

NOTE: You can turn off this warning by setting the MCA parameter
      btl_openib_warn_no_device_params_found to 0.
--------------------------------------------------------------------------
--------------------------------------------------------------------------
No OpenFabrics connection schemes reported that they were able to be
used on a specific port.  As such, the openib BTL (OpenFabrics
support) will be disabled for this port.

  Local host:           130-61-76-112
  Local device:         mlx5_0
  Local port:           1
  CPCs attempted:       udcm
-------------

In [29]:
df_neg_text = pd.DataFrame(neg_text_list, columns = ['text'])
# print(df_neg_text)
df_neg_text_small = df_neg_text[:8]
# print(df_neg_text_small)

In [30]:
with open("test_neg_loop.txt", "w") as f_out:
    f_out.write("\n".join(df_neg_text["text"]))

In [31]:
# !export TRAIN_FILE=/content/FineTuneText/ImdbPos.txt
#     --overwrite_cache \
!python  /home/ubuntu/pytorch-transformers/examples/run_lm_finetuning.py \
    --output_dir=ImdbNeg_model \
    --model_type=gpt2 \
    --model_name_or_path=gpt2 \
    --do_train \
    --learning_rate=2e-4 \
    --save_steps=116 \
    --warmup_steps=50 \
    --gradient_accumulation_steps=1 \
    --overwrite_output_dir \
    --train_data_file=/home/ubuntu/test_neg_loop.txt \
    --num_train_epochs=10

--------------------------------------------------------------------------
detected:

  Local host:            130-61-76-112
  Device name:           mlx5_0
  Device vendor ID:      0x02c9
  Device vendor part ID: 4122

Default device parameters will be used, which may result in lower
performance.  You can edit any of the files specified by the
btl_openib_device_param_files MCA parameter to set values for your
device.

NOTE: You can turn off this warning by setting the MCA parameter
      btl_openib_warn_no_device_params_found to 0.
--------------------------------------------------------------------------
--------------------------------------------------------------------------
No OpenFabrics connection schemes reported that they were able to be
used on a specific port.  As such, the openib BTL (OpenFabrics
support) will be disabled for this port.

  Local host:           130-61-76-112
  Local device:         mlx5_0
  Local port:           1
  CPCs attempted:       udcm
-------------

In [32]:
# /home/ubuntu/pytorch-transformers/examples
# /home/ubuntu/FineTuneText/ImdbPos.txt

In [33]:
# pos_model_dict = torch.load('/home/ubuntu/ImdbPOS_model/pytorch_model.bin', map_location='cuda') 
# #ImdbPOS_model/pytorch_model.bin
# #/home/ubuntu/ImdbPOS_model

In [34]:
print(type(pos_model_dict))

NameError: name 'pos_model_dict' is not defined

In [ ]:
# from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel, AutoModel
# import torch
# head_model_pos = GPT2LMHeadModel.from_pretrained('gpt2', state_dict=pos_model_dict)
# base_gpt2_head = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
# text_ids = base_tokenizer.encode(text, return_tensors = 'pt')
# text_ids